In [60]:
import pandas as pd
from IPython.display import display
import pickle

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tensorflow.keras import models

In [19]:
class CFG:
    dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
    n_folds=4
    target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']

In [7]:
train=pd.read_csv(f"{CFG.dataset_dir}train.csv")
display(train.columns)

Index(['StudyInstanceUID', 'ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present',
       'PatientID'],
      dtype='object')

In [32]:
cvc_cols=[col_name for col_name in CFG.target_cols if ("CVC" in col_name)]

train["CVC Present"]=train[cvc_cols].sum(axis=1)
false_idx=(train["CVC Present"]==0)
train["CVC Present"]=1
train.loc[false_idx,"CVC Present"]=0
display(train["CVC Present"].value_counts())

1    29333
0      750
Name: CVC Present, dtype: int64

In [39]:
fold=train.copy()

skf=StratifiedKFold(n_splits=CFG.n_folds)
for n,(train_idx,val_idx) in enumerate(skf.split(train,train["CVC Present"])):
    fold.loc[val_idx,"folds"]=n
fold["folds"]=fold["folds"].astype(int)

display(fold)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,CVC Present,folds
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1,0,0
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c,1,0
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5,1,0
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814,1,0
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0,0,1,0,0,0,0,0,1,1,0,5b5b9ac30,1,3
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,0,1,0,7192404d8,1,3
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0,0,1,0,0,1,0,1,0,1,0,d4d1b066d,1,3
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0,0,0,0,0,0,0,0,1,0,0,01a6602b8,1,3


In [35]:
features=pd.read_csv("./models/efficientnet_output_normalized.csv")
display(features)

,StudyInstanceUID,PatientID,0,1,2,3,4,5,6,7,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,ec89415d1,0.169359,0.324016,0.173553,0.095382,0.063119,0.165738,0.181906,0.437814,...,0.302277,0.070963,0.354162,0.284205,0.057408,0.101549,0.200983,0.098618,0.276763,0.473490
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,bf4c6da3c,0.365546,0.175262,0.286057,0.205560,0.388747,0.452293,0.021495,0.460131,...,0.232927,0.223938,0.135535,0.042117,0.169312,0.383767,0.095033,0.151809,0.048868,0.117311
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,3fc1c97e5,0.211337,0.303350,0.184402,0.248883,0.202594,0.301177,0.208046,0.416033,...,0.370746,0.315548,0.384991,0.181793,0.134341,0.147335,0.313144,0.356484,0.112562,0.404958
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,c31019814,0.520994,0.385111,0.367907,0.247489,0.097945,0.448468,0.377076,0.108094,...,0.355882,0.156018,0.316916,0.141020,0.041345,0.297808,0.273713,0.329452,0.203884,0.219169
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,207685cd1,0.383775,0.214603,0.263212,0.429333,0.619226,0.365313,0.264323,0.201577,...,0.302169,0.318546,0.097627,0.337127,0.124034,0.266738,0.595066,0.150074,0.454537,0.071115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,5b5b9ac30,0.152407,0.273989,0.429204,0.095627,0.143194,0.151159,0.190642,0.151711,...,0.417215,0.330231,0.257251,0.193722,0.070763,0.415085,0.388353,0.156808,0.239679,0.161568
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,7192404d8,0.303968,0.299640,0.232293,0.115276,0.309395,0.338309,0.164800,0.221858,...,0.509008,0.143263,0.214838,0.167794,0.075009,0.163606,0.415589,0.059742,0.148673,0.164074
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,d4d1b066d,0.316939,0.351076,0.589629,0.291154,0.185748,0.232997,0.113028,0.353113,...,0.542412,0.236196,0.170918,0.283692,0.270579,0.275420,0.285688,0.203561,0.118533,0.222272
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,01a6602b8,0.239194,0.393650,0.438603,0.363903,0.144577,0.259132,0.366278,0.193645,...,0.387640,0.075916,0.316283,0.119015,0.101108,0.645644,0.305596,0.319096,0.330187,0.278469


In [56]:
autoencoder=models.load_model("./models/autoencoder_splits10/")

layer_name="dense_1"
hidden_layer_model=models.Model(inputs=autoencoder.input,outputs=autoencoder.get_layer(layer_name).output)

pred=hidden_layer_model.predict(features.iloc[:,2:])
X=pd.DataFrame(pred)
display(X)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,3.523285,2.151111,3.073467,3.363639,3.601960,1.177225,0.0,3.399705,4.387469,...,0.0,1.328145,1.331692,1.144717,2.307059,2.733480,2.879669,4.131839,3.130108,2.915718
1,0.0,2.752369,2.418850,2.875795,3.922628,5.291559,4.010684,0.0,4.176135,4.387034,...,0.0,1.365769,3.142598,6.856853,2.623820,4.440138,6.352470,7.112836,8.021735,6.673033
2,0.0,5.127551,4.384488,8.046625,5.567223,6.195307,7.095205,0.0,4.194753,1.949514,...,0.0,1.316996,5.129144,5.434055,5.875688,4.957835,5.093932,4.814246,1.765945,7.694550
3,0.0,2.767875,1.731207,5.098400,2.484329,3.534876,4.957430,0.0,6.703936,4.863359,...,0.0,0.875055,3.810669,2.164724,1.225728,1.915580,2.081527,1.964060,2.458131,4.171708
4,0.0,5.906751,2.528070,5.296381,2.448073,2.876235,8.141816,0.0,4.746807,5.582016,...,0.0,1.980754,6.254673,1.629905,3.840276,2.863812,4.631470,4.077279,5.729913,6.588160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,0.0,2.509894,2.170190,3.057890,5.210413,4.008926,4.459599,0.0,4.928145,5.961706,...,0.0,2.131990,4.075906,4.149724,2.756427,2.651626,5.739603,5.785378,3.671273,3.416882
30079,0.0,3.314908,2.372607,1.440810,2.099609,3.282451,4.099072,0.0,3.791726,5.290777,...,0.0,1.977184,6.820914,5.900064,3.299438,3.492918,4.771842,4.394142,3.231114,3.616853
30080,0.0,5.604525,2.085924,4.015664,4.236988,6.777108,4.505717,0.0,5.942400,6.121769,...,0.0,1.665230,5.557427,4.285787,2.033923,4.248517,4.583074,10.027946,7.137775,5.121286
30081,0.0,5.239021,1.398003,3.633277,4.615515,3.244748,3.063015,0.0,6.512488,2.196148,...,0.0,1.476729,3.201416,1.321704,2.968237,3.272295,2.854025,0.896478,3.066891,2.736391


In [59]:
%%time

from optuna.integration import lightgbm as lgb

n=0
train_idx=(fold["folds"]!=n)
val_idx=(fold["folds"]==n)

def optimize_params():
    X_train=X[train_idx]
    X_val=X[val_idx]
    y_train=train["CVC Present"][train_idx]
    y_val=train["CVC Present"][val_idx]

    lgb_train=lgb.Dataset(X_train,label=y_train)
    lgb_test=lgb.Dataset(X_val,label=y_val,reference=lgb_train)
    
    params={
        "task":"train",
        "boosting_type":"gbdt",
        "objective":"binary",
        "metric":"auc",
        "learning_rate":0.1,
        "num_iterations":100
        # "early_stopping_rounds":200, #early_stopping_roundsを指定しないとbest_iterationは保存されない
    }

    opt=lgb.train(params,lgb_train,valid_sets=lgb_test, verbose_eval=50)
    pickle.dump(opt.params,open(f"{models_dir}lgbm_cvc/lgb_params_{n}.pickle","wb"))

optimize_params()

e: 0.7385304264313755 and parameters: {'lambda_l1': 3.2400178476066615e-06, 'lambda_l2': 0.00013353527786452612}. Best is trial 43 with value: 0.7453032197788487.
regularization_factors, val_score: 0.745303:  30%|###       | 6/20 [00:20<00:48,  3.43s/it][100]	valid_0's auc: 0.73853
[LightGBM] [Info] Number of positive: 22000, number of negative: 562
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011502 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20151
[LightGBM] [Info] Number of data points in the train set: 22562, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.975091 -> initscore=3.667296
[LightGBM] [Info] Start training from score 3.667296
[50]	valid_0's auc: 0.733921
regularization_factors, val_score: 0.745303:  35%|###5      | 7/20 [00:23<00:45,  3.52s/it][100]	valid_0's auc: 0.736295
[LightGBM] [Info] Number of positive: 22000, number of negative: 562
[Lig

NameError: name 'pickle' is not defined

In [55]:
import lightgbm


def get_pred(train,val):

    X_train,y_train=train
    X_val,y_val=val

    lgb_train=lightgbm.Dataset(X_train,label=y_train)
    lgb_test=lightgbm.Dataset(X_val,label=y_val,reference=lgb_train)

    # params=pickle.load(open(f"{CFG.models_dir}autoencoder_smallLR/lgb_params_{col_index}.pickle","rb"))
    # params["early_stopping_rounds"]=500
    params={
        "task":"train",
        "boosting_type":"gbdt",
        "objective":"binary",
        "metric":"auc",
        "early_stopping_rounds":50, #early_stopping_roundsを指定しないとbest_iterationは保存されない
    }

    lgbm_model=lightgbm.train(params,lgb_train,valid_sets=lgb_test,verbose_eval=10)
    pred=lgbm_model.predict(X_val)
    auc=roc_auc_score(y_val,pred)

    return pred,auc


results=pd.DataFrame(columns=["CVC Present"])

for n in range(CFG.n_folds):
    print(f"\nfold - {n}")
    train_idx=(fold["folds"]!=n)
    val_idx=(fold["folds"]==n)

    train_data=X[train_idx].iloc[:,1:],train["CVC Present"][train_idx]
    val_data=X[val_idx].iloc[:,1:],train["CVC Present"][val_idx]

    _,auc=get_pred(train=train_data,val=val_data)
        
    results.loc[f"fold - {n}","CVC Present"]=auc


display(results,results.mean(axis=0))


fold - 0
C:\Users\kamim\anaconda3\envs\py38\lib\site-packages\lightgbm\engine.py:156: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[LightGBM] [Info] Number of positive: 22000, number of negative: 562
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014057 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20145
[LightGBM] [Info] Number of data points in the train set: 22562, number of used features: 79
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.975091 -> initscore=3.667296
[LightGBM] [Info] Start training from score 3.667296
Training until validation scores don't improve for 50 rounds
[10]	valid_0's auc: 0.703537
[20]	valid_0's auc: 0.703236
[30]	valid_0's auc: 0.71102
[40]	valid_0's auc: 0.716908
[50]	valid_0's auc: 0.721972
[60]	valid_0's auc: 0.721623
[70]	valid_0

,CVC Present
fold - 0,0.738053
fold - 1,0.757254
fold - 2,0.756679
fold - 3,0.749063


CVC Present    0.750262
dtype: float64